In [1]:
from bastionlab import Connection, Identity
import polars as pl

In [3]:
data_owner = Identity.create("data_owner")
client = Connection("localhost").client

In [4]:
target_names = ["setosa", "versicolor", "virginica"]
df = pl.read_csv(
    "iris.csv",
    has_header=False,
    new_columns=[
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)",
        "target",
    ],
)


df = df.with_column(
    pl.col("target")
    .map(lambda a: a.apply(lambda b: True if target_names[b] == "virginica" else False))
    .alias("target_class")
)

target_class_df = pl.DataFrame({"classes": target_names})
target_class_df = target_class_df.with_row_count("id")

In [5]:
rdf = client.polars.send_df(df)
classes_rdf = client.polars.send_df(target_class_df)

train_rdf = rdf.select(pl.col(["petal length (cm)", "petal width (cm)"])).collect()
test_rdf = rdf.select(pl.col("target_class")).collect()
print(train_rdf, test_rdf)

FetchableLazyFrame(identifier=013c99de-50e9-4e55-94c1-56285245b423) FetchableLazyFrame(identifier=90e69b48-cac1-4465-9929-a7d79e5f0b94)


In [6]:
from bastionlab.linfa.trainers import LogisticRegression

trainer = client.linfa.train(train_rdf, test_rdf, LogisticRegression())

AttributeError: 'Client' object has no attribute 'train'

In [53]:
res = client.predict(trainer, [5.5, 4.2])
print(res)

FetchableLazyFrame(identifier=8cd288f3-040a-41fb-9f5a-c98817a555a9)
